<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/pyomo_tut3_lp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For colab, install the necessary kits
!pip install pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

Powerco has 3 electric power plants that supply the needs of 4 cities. Each power plant can supply the following numbers of kWh of electricity.   
Plant 1 -- 35 million;  
Plant 2 -- 50 million;  
Plant 3 -- 40 million;  
The peak power demands in these cities are:
City 1 -- 45 million;  
City 2 -- 20 million;  
City 3 -- 30 million;  
City 4 -- 30 million;  
The cost of sending 1 million kWh of electricity from plant to city can be found below.  Formulate LP to minimize the cost of meeting each city's peak power demand.  
|      | TO |   |   |   | Supply  |  
|------|----|---|---|---|---|  
|   From|  city 1  |  city 2 | city 3  | city 4  |(Million kWh)|  
|Plant 1|  $8      |  $6     |  $10    |  $9     |   35        |   
|Plant 2|  $9      |  $12    |  $13    |  $7     |   50        |   
|Plant 3|  $14     |  $9     |  $16    |  $5     |   40        |   
|Demand |  45      |  20     |  30     |  30     |             |   

First, decide on the decision variables for the problems.  
The decision vraibles are the power to each city from each plant.  
There need to be 2 sets of indices. 1 set representing the plant, the other representing the cities.  
Let E be the ammount of energy in million kWh.  
Let C be the cost of sending the energy in dollars.
Let p be the index for power plants.
Let c be the index for cities.
$$  
\\p = \{1, 2, 3\}  
\\c = \{1, 2, 3, 4 \}  
\\obj:~~~~min~sum(E_{p,c} \times C_{c,p})
\\s.t:
\\sum(E_{1,c})\leq35
\\sum(E_{2,c})\leq50
\\sum(E_{3,c})\leq40
\\sum(E_{p,1})\geq45
\\sum(E_{p,2})\geq20
\\sum(E_{p,3})\geq30
\\sum(E_{p,4})\geq30
\\E_{p,c}\geq0
$$  

In [2]:
# Import the tools
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [3]:
from pyomo.core.base import initializer
# Create the model
model = pyo.ConcreteModel()

# Create the sets for indicies
# Index for power plants
model.p = pyo.Set(initialize=['Plant1', 'Plant2', 'Plant3'])
# Index for cities
model.c = pyo.Set(initialize=['City1', 'City2', 'City3', 'City4'])

In [ ]:
# Create the parameters, parameters are stuff that should be fixed
# for the duration of the calculation by optimisation
model.energy = pyo.Param()

# Define the limit for the ammount of energy each plant can produce, this is a parameter
plant_cap_dict = {'Plant1':35, 'Plant2':50, 'Plant3':40}
model.plant_cap = pyo.Param(model.p, initialize=plant_cap_dict)
plant_cap = model.plant_cap

# Define the ammount of energy each city will require, this is a parameter.
city_demand_dict = {'City1':45, 'City2':20, 'City3':30, 'City4':30}
model.city_demand = pyo.Param(model.c, initialize=city_demand_dict)
city_demand = model.city_demand

# Define the cost of sending energy from plant to city.
# Index needed is both p and c
price_dict = {
    ('Plant1', 'City1'):8,
    ('Plant1', 'City2'):8,
    ('Plant1', 'City3'):8,
    ('Plant1', 'City4'):8,
    ('Plant2', 'City1'):8,
    ('Plant2', 'City2'):8,
    ('Plant2', 'City3'):8,
    ('Plant2', 'City4'):8,
    ('Plant3', 'City1'):8,
    ('Plant3', 'City2'):8,
    ('Plant3', 'City3'):8,
    ('Plant3', 'City4'):8
}
model.price = pyo.Param(model.p, model.c, initialize=price_dict)